In [4]:
import math
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np
import os

# -------------------------------------------------------------------------
# Helper Functions
# -------------------------------------------------------------------------
def enhance_xray(frame):
    """
    Example enhancement for an X-ray (grayscale) image using CLAHE.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_gray = clahe.apply(gray)
    return cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)

def morphological_segmentation(frame):
    """
    Simple morphological segmentation (e.g., 'open' to remove noise).
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    morphed = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    return cv2.cvtColor(morphed, cv2.COLOR_GRAY2BGR)

def apply_colormap(frame, colormap=cv2.COLORMAP_JET):
    """
    Apply a colormap, default is JET.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.applyColorMap(gray, colormap)

def compress_image_and_save(frame, save_path, quality=70):
    """
    Save the given BGR frame as a JPEG with specified compression quality.
    """
    cv2.imwrite(save_path, frame, [cv2.IMWRITE_JPEG_QUALITY, quality])

def cv_to_tkimage(cv_image, target_width=300):
    """
    Convert OpenCV BGR image -> Tkinter PhotoImage
    (with resizing and aspect ratio preservation).
    """
    if cv_image is None:
        return None
    height, width, _ = cv_image.shape
    aspect_ratio = width / height
    new_width = target_width
    new_height = int(new_width / aspect_ratio)

    resized = cv2.resize(cv_image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb)
    return ImageTk.PhotoImage(pil_img)

# -------------------------------------------------------------------------
# Secondary Window Class
# -------------------------------------------------------------------------
class SelectedFramesWindow(tk.Toplevel):
    """
    A Toplevel window that displays all selected frames
    on top of a background image (behind them),
    with 6 images per row in a smaller size (100px),
    and added spacing between them.
    """
    def __init__(self, master, selected_frames, style=None):
        super().__init__(master)
        self.title("Process Selected Frames")
        self.geometry("900x600")

        # Store frames (OpenCV format)
        self.frames = [f.copy() for f in selected_frames]

        # Keep references to Tk PhotoImages so they're not GC'd
        self.tk_images = []

        # Attempt to load a background image
        self.bg_img = None
        try:
            bg_pil = Image.open("background.jpg").convert("RGBA")
            bg_pil = bg_pil.resize((900, 600), Image.ANTIALIAS)
            self.bg_img = ImageTk.PhotoImage(bg_pil)
        except Exception:
            pass

        # Create a Canvas that fills the entire window
        self.canvas = tk.Canvas(self, width=900, height=600, highlightthickness=0)
        self.canvas.pack(fill=tk.BOTH, expand=True)

        # Place the background image (if loaded)
        if self.bg_img:
            self.canvas.create_image(0, 0, anchor="nw", image=self.bg_img)

        # -- Create button widgets on the Canvas (absolute positioning) --
        self.enhance_btn = ttk.Button(self, text="Enhance (X-ray)", command=self.enhance_selected)
        self.segment_btn = ttk.Button(self, text="Segment", command=self.segment_selected)
        self.color_btn   = ttk.Button(self, text="Coloring", command=self.color_selected)
        self.save_btn    = ttk.Button(self, text="Save All", command=self.save_all_frames)

        # Place them on the canvas
        self.canvas.create_window(20, 20, anchor="nw", window=self.enhance_btn)
        self.canvas.create_window(140, 20, anchor="nw", window=self.segment_btn)
        self.canvas.create_window(240, 20, anchor="nw", window=self.color_btn)
        self.canvas.create_window(330, 20, anchor="nw", window=self.save_btn)

        # Display the frames (6 images per row, 100px each, with spacing)
        self.display_frames()

        # Bind a resize event (optional)
        self.bind("<Configure>", self.on_resize)

    def on_resize(self, event):
        """
        Re-draw background if window is resized (if desired).
        We skip re-scaling the background for simplicity.
        """
        pass

    def display_frames(self):
        """
        Display the frames as canvas images on top of the background.
        We'll do 6 images per row, each 100px wide,
        with extra spacing between them.
        """
        self.tk_images.clear()

        num_cols = 6
        x_offset = 10   # left margin
        y_offset = 70   # top margin (below the buttons)
        spacing_x = 20  # horizontal gap (increased)
        spacing_y = 20  # vertical gap (increased)
        thumb_width = 100  # Each frame's display width

        for i, frame in enumerate(self.frames):
            row = i // num_cols
            col = i % num_cols
            x_pos = x_offset + col * (thumb_width + spacing_x)
            # approximate height by reusing thumb_width
            y_pos = y_offset + row * (thumb_width + spacing_y)

            tk_img = cv_to_tkimage(frame, target_width=thumb_width)
            self.canvas.create_image(x_pos, y_pos, anchor="nw", image=tk_img)
            self.tk_images.append(tk_img)  # keep a reference to avoid GC

    def enhance_selected(self):
        """Apply X-ray enhancement to all frames."""
        self.frames = [enhance_xray(f) for f in self.frames]
        self.display_frames()

    def segment_selected(self):
        """Apply morphological segmentation to all frames."""
        self.frames = [morphological_segmentation(f) for f in self.frames]
        self.display_frames()

    def color_selected(self):
        """Apply a colormap to all frames."""
        self.frames = [apply_colormap(f) for f in self.frames]
        self.display_frames()

    def save_all_frames(self):
        """Let user pick a folder to save all frames."""
        directory = filedialog.askdirectory(title="Select Folder to Save Frames")
        if not directory:
            return
        for i, frame in enumerate(self.frames):
            save_path = os.path.join(directory, f"selected_frame_{i+1}.jpg")
            compress_image_and_save(frame, save_path, quality=70)
        messagebox.showinfo("Done", f"Saved {len(self.frames)} frames to {directory}.")

# -------------------------------------------------------------------------
# Main Application Class
# -------------------------------------------------------------------------
class FrameSelectorApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Pro Video Frame Selector & Editor")
        self.geometry("1300x800")
        self.minsize(1100, 600)

        # State variables
        self.cap = None
        self.total_frames = 0
        self.current_frame_index = 0
        self.current_frame = None
        self.tk_images = []
        self.sampled_frames = []
        self.sampled_thumbnails = []
        self.thumbnail_labels = []

        # Number of frames to sample
        self.num_frames_to_sample = tk.IntVar(value=30)

        # Let user specify how many frames they can select (max)
        self.max_frames_to_open = tk.IntVar(value=6)  # up to 6 frames by default

        self.selected_indices = []

        self.setup_ui()

    def setup_ui(self):
        # Menu bar
        menu_bar = tk.Menu(self)
        self.config(menu=menu_bar)
        file_menu = tk.Menu(menu_bar, tearoff=False)
        file_menu.add_command(label="Load Video", command=self.load_video)
        file_menu.add_command(label="Exit", command=self.quit)
        menu_bar.add_cascade(label="File", menu=file_menu)

        # TOP FRAME
        top_frame = ttk.Frame(self, padding=10)
        top_frame.pack(side=tk.TOP, fill=tk.X)

        # Buttons row
        btn_row = ttk.Frame(top_frame)
        btn_row.pack(side=tk.TOP, fill=tk.X)

        browse_btn = ttk.Button(btn_row, text="Load Video", command=self.load_video)
        browse_btn.pack(side=tk.LEFT, padx=5)

        show_btn = ttk.Button(btn_row, text="Show Frame", command=self.show_selected_frame)
        show_btn.pack(side=tk.LEFT, padx=5)

        enhance_btn = ttk.Button(btn_row, text="Enhance (X-ray)", command=self.enhance_frame)
        enhance_btn.pack(side=tk.LEFT, padx=5)

        segment_btn = ttk.Button(btn_row, text="Segment", command=self.segment_frame)
        segment_btn.pack(side=tk.LEFT, padx=5)

        color_btn = ttk.Button(btn_row, text="Coloring", command=self.color_frame)
        color_btn.pack(side=tk.LEFT, padx=5)

        save_btn = ttk.Button(btn_row, text="Save Frame", command=self.save_current_frame)
        save_btn.pack(side=tk.LEFT, padx=5)

        # SECOND ROW: sampling controls + frame slider
        second_row = ttk.Frame(top_frame)
        second_row.pack(side=tk.TOP, fill=tk.X, pady=(10, 0))

        # Left side: sampling
        sampling_frame = ttk.Frame(second_row)
        sampling_frame.pack(side=tk.LEFT, fill=tk.X)

        ttk.Label(sampling_frame, text="Number of frames to sample: ").pack(side=tk.LEFT, padx=5)
        spin_sample = ttk.Spinbox(sampling_frame, from_=1, to=500,
                                  textvariable=self.num_frames_to_sample, width=5)
        spin_sample.pack(side=tk.LEFT, padx=5)

        sample_btn = ttk.Button(sampling_frame, text="Sample Frames", command=self.sample_frames)
        sample_btn.pack(side=tk.LEFT, padx=5)

        # Spinbox for max frames to open
        ttk.Label(sampling_frame, text="Max frames to open: ").pack(side=tk.LEFT, padx=20)
        spin_max = ttk.Spinbox(sampling_frame, from_=1, to=20,
                               textvariable=self.max_frames_to_open, width=5)
        spin_max.pack(side=tk.LEFT, padx=5)

        # Button to open new window with selected frames
        self.open_selected_btn = ttk.Button(
            sampling_frame, text="Open Selected Frames",
            command=self.open_selected_frames_window,
            state=tk.DISABLED
        )
        self.open_selected_btn.pack(side=tk.LEFT, padx=20)

        # Right side: frame slider
        slider_frame = ttk.Frame(second_row)
        slider_frame.pack(side=tk.RIGHT, fill=tk.X, padx=5)

        self.frame_slider = tk.Scale(slider_frame, from_=0, to=0,
                                     orient=tk.HORIZONTAL, label="Frame Index",
                                     length=300, command=self.slider_changed)
        self.frame_slider.pack(side=tk.RIGHT, padx=5)

        # MAIN DISPLAY
        self.display_frame = ttk.Frame(self, padding=10)
        self.display_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # THUMBNAILS DISPLAY
        self.thumbnails_frame = ttk.Frame(self, padding=10)
        self.thumbnails_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

        ttk.Label(self.thumbnails_frame,
                  text="Sampled Frames (Click to Select):",
                  font=("Arial", 12, "bold")
        ).pack(side=tk.TOP, pady=5)

        self.canvas = tk.Canvas(self.thumbnails_frame)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.scrollbar = ttk.Scrollbar(self.thumbnails_frame, orient=tk.VERTICAL,
                                       command=self.canvas.yview)
        self.scrollbar.pack(side=tk.LEFT, fill=tk.Y)

        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.thumbnail_container = ttk.Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.thumbnail_container, anchor="nw")
        self.thumbnail_container.bind("<Configure>",
                                      lambda e: self.canvas.configure(
                                          scrollregion=self.canvas.bbox("all")))

    # ---------------------------------------------------------------------
    # Video Loading & Frame Navigation
    # ---------------------------------------------------------------------
    def load_video(self):
        video_path = filedialog.askopenfilename(
            title="Select a video file",
            filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv"), ("All files", "*.*")]
        )
        if not video_path:
            return

        if self.cap is not None:
            self.cap.release()

        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            messagebox.showerror("Error", "Cannot open selected video.")
            return

        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.frame_slider.config(to=self.total_frames - 1)
        self.frame_slider.set(0)
        self.current_frame_index = 0
        self.show_selected_frame()
        self.clear_sampled_frames()

    def slider_changed(self, value):
        self.current_frame_index = int(float(value))

    def show_selected_frame(self):
        if not self.cap:
            return
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, self.current_frame_index)
        ret, frame = self.cap.read()
        if ret:
            self.current_frame = frame.copy()
            self.display_current_frame()

    def display_current_frame(self):
        for child in self.display_frame.winfo_children():
            child.destroy()
        self.tk_images.clear()

        if self.current_frame is None:
            return

        tk_image = cv_to_tkimage(self.current_frame, target_width=300)
        lbl = ttk.Label(self.display_frame, image=tk_image,
                        borderwidth=2, relief="groove")
        lbl.pack(side=tk.TOP, pady=10)
        lbl.image = tk_image
        self.tk_images.append(tk_image)

    # ---------------------------------------------------------------------
    # Processing "current frame"
    # ---------------------------------------------------------------------
    def enhance_frame(self):
        if self.current_frame is not None:
            self.current_frame = enhance_xray(self.current_frame)
            self.display_current_frame()

    def segment_frame(self):
        if self.current_frame is not None:
            self.current_frame = morphological_segmentation(self.current_frame)
            self.display_current_frame()

    def color_frame(self):
        if self.current_frame is not None:
            self.current_frame = apply_colormap(self.current_frame)
            self.display_current_frame()

    def save_current_frame(self):
        if self.current_frame is None:
            return
        save_path = filedialog.asksaveasfilename(
            title="Save Current Frame",
            defaultextension=".jpg",
            filetypes=[("JPEG files", "*.jpg"), ("All files", "*.*")]
        )
        if save_path:
            compress_image_and_save(self.current_frame, save_path, quality=70)
            print(f"Frame saved as: {save_path}")

    # ---------------------------------------------------------------------
    # Sampling Frames & Thumbnails
    # ---------------------------------------------------------------------
    def sample_frames(self):
        if not self.cap or self.total_frames == 0:
            messagebox.showwarning("No Video", "Please load a valid video first.")
            return

        self.clear_sampled_frames()
        n = self.num_frames_to_sample.get()
        if n < 1:
            return

        interval = max(self.total_frames // n, 1)
        self.sampled_frames = []
        for i in range(n):
            idx = i * interval
            if idx >= self.total_frames:
                break
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = self.cap.read()
            if ret:
                self.sampled_frames.append(frame.copy())

        self.display_thumbnails()

    def display_thumbnails(self):
        for child in self.thumbnail_container.winfo_children():
            child.destroy()
        self.sampled_thumbnails.clear()
        self.thumbnail_labels.clear()
        self.selected_indices.clear()

        self.open_selected_btn.config(state=tk.DISABLED)

        columns = 5
        row = 0
        col = 0

        for i, frame in enumerate(self.sampled_frames):
            tk_img = cv_to_tkimage(frame, target_width=60)
            thumb_label = ttk.Label(self.thumbnail_container,
                                    image=tk_img,
                                    borderwidth=2,
                                    relief="groove")
            thumb_label.image = tk_img
            thumb_label.grid(row=row, column=col, padx=5, pady=5)

            self.sampled_thumbnails.append(tk_img)
            self.thumbnail_labels.append(thumb_label)

            thumb_label.bind("<Button-1>", lambda e, idx=i: self.on_thumbnail_click(idx))

            col += 1
            if col >= columns:
                col = 0
                row += 1

    def on_thumbnail_click(self, idx):
        # Toggle selection
        if idx in self.selected_indices:
            # Unselect
            self.selected_indices.remove(idx)
            self.thumbnail_labels[idx].configure(borderwidth=2, relief="groove")
        else:
            # Check maximum
            if len(self.selected_indices) < self.max_frames_to_open.get():
                self.selected_indices.append(idx)
                self.thumbnail_labels[idx].configure(borderwidth=3, relief="solid")
            else:
                messagebox.showinfo(
                    "Selection Limit",
                    f"You can only select up to {self.max_frames_to_open.get()} frames."
                )

        # Enable the "Open Selected Frames" button if there's at least 1 selected
        if len(self.selected_indices) > 0:
            self.open_selected_btn.config(state=tk.NORMAL)
        else:
            self.open_selected_btn.config(state=tk.DISABLED)

    def clear_sampled_frames(self):
        self.sampled_frames.clear()
        for child in self.thumbnail_container.winfo_children():
            child.destroy()
        self.sampled_thumbnails.clear()
        self.thumbnail_labels.clear()
        self.selected_indices.clear()
        self.open_selected_btn.config(state=tk.DISABLED)

    # ---------------------------------------------------------------------
    # Opening the second window with all selected frames
    # ---------------------------------------------------------------------
    def open_selected_frames_window(self):
        if len(self.selected_indices) == 0:
            messagebox.showerror("Error", "No frames selected.")
            return
        # Gather selected frames
        selected_frames = [self.sampled_frames[i] for i in self.selected_indices]

        # Create the second window
        SelectedFramesWindow(self, selected_frames)

# -------------------------------------------------------------------------
# Run the Application
# -------------------------------------------------------------------------
def main():
    app = FrameSelectorApp()
    app.mainloop()

if __name__ == "__main__":
    main()
